In [ ]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.4
sess = tf.Session(config=config)
import numpy as np 
import pandas as pd
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.callbacks import EarlyStopping
from keras.utils import plot_model
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from utils import plot_training
%matplotlib inline

In [ ]:
datagen = ImageDataGenerator(rescale=1./255)
train_generator = datagen.flow_from_directory(
        'data/train/',  
        batch_size=1,
        class_mode='categorical')

In [ ]:
# let's have a look at the images
x, y = train_generator.next()
plt.imshow((x[0]*255).astype('uint8'));
print(list(train_generator.class_indices.keys())[np.argmax(y)])

In [ ]:
# X_data, Y_data = [], []
# for _ in tqdm(range(2750)):
#     x, y = train_generator.next()
#     X_data.append(x[0])
#     Y_data.append(y[0])
# X_data = np.asarray(X_data)
# Y_data = np.asarray(Y_data)
# np.save('data/X_data.npy', X_data)
# np.save('data/Y_data.npy', Y_data)

In [ ]:
X_data = np.load('data/X_data.npy')
Y_data = np.load('data/Y_data.npy')

In [ ]:
def get_model():
    input_img = Input((256, 256, 3))
    X = BatchNormalization()(input_img)
    X = Convolution2D(16, (3, 3), activation='relu')(X)
    X = BatchNormalization()(X)
    X = Convolution2D(16, (3, 3), activation='relu')(X)
    X = MaxPooling2D()(X)
    X = Convolution2D(32, (3, 3), activation='relu')(X)
    X = BatchNormalization()(X)
    X = Convolution2D(32, (3, 3), activation='relu')(X)
#     X = GlobalMaxPooling2D()(X)
    X = Flatten()(X)
    X = BatchNormalization()(X)
    X = Dense(512, activation='relu')(X)
    X = Dropout(0.2)(X)
    X = Dense(10, activation='softmax')(X)
    model = Model(inputs=input_img, outputs=X)

    model.compile(optimizer='adam', loss='categorical_crossentropy', 
                  metrics=['acc'])
#     model.summary()
    return model

In [ ]:
model = get_model()

In [ ]:
model_history = model.fit(X_data, Y_data, batch_size=300, epochs=10, validation_split=0.2,
                          callbacks=[EarlyStopping(monitor='val_acc', patience=3, verbose=1)])

In [ ]:
plot_training(model_history)

In [ ]:
# load test images
X_test = []
sub = pd.read_csv('data/sample_submission.csv')

for fname in tqdm(sub['fname']):
    filepath = 'data/test/' + fname
    X_test.append(img_to_array(load_img(filepath, target_size=(256, 256))))
X_test = np.asarray(X_test)

In [ ]:
classes = ['HTC-1-M7', 'LG-Nexus-5x', 'Motorola-Droid-Maxx', 'Motorola-Nexus-6', 'Motorola-X', 
           'Samsung-Galaxy-Note3', 'Samsung-Galaxy-S4','Sony-NEX-7','iPhone-4s','iPhone-6']
preds = model.predict(X_test, verbose=1)
preds = np.argmax(preds, axis=1)
preds = [classes[p] for p in tqdm(preds)]

In [ ]:
sub['camera'] = preds
sub.to_csv('sub.csv', index=False)

In [ ]:
sub.head()